In [1]:
import pandas as pd
passengers = pd.read_csv("train.csv")
print(passengers.shape)
print(passengers.head())

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

In [2]:
passengers['Sex'] = passengers['Sex'].map({'female':1,'male':0})
passengers['Age'].fillna(value=passengers['Age'].mean(), inplace=True)

In [3]:
passengers['FirstClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 1 else 0)
passengers['SecondClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 2 else 0)

In [4]:
features = passengers[['Sex', 'Age', 'FirstClass', 'SecondClass']]
survival = passengers['Survived']

In [6]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, survival)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [8]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train_features, train_labels)

LogisticRegression()

In [9]:
print(model.score(train_features, train_labels))

0.7904191616766467


In [10]:
from art.attacks.evasion import DeepFool
from art.estimators.classification.scikitlearn import SklearnClassifier

classifier = SklearnClassifier(model=model, clip_values=(0, 1))
attack = DeepFool(classifier)
x_test_adv = attack.generate(train_features)

DeepFool:   0%|          | 0/668 [00:00<?, ?it/s]

In [11]:
print(model.score(x_test_adv, train_labels))
print(model.score(train_features, train_labels))

0.20808383233532934
0.7904191616766467


In [12]:
from art.defences.trainer import AdversarialTrainer

# ART 에서 제공되는 Adversarial Trainer 선언한다

trainer = AdversarialTrainer(
    classifier,
    attack,
    ratio=1.0
)

In [13]:
# 위에서 선언한 Trainer도 같은 데이터 셋을 이용하여 학습시켜준다.
trainer.fit(train_features, train_labels, batch_size=64, nb_epochs=3)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/3 [00:00<?, ?it/s]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([366, 354, 319,  64, 659,\n            ...\n            467, 507, 485, 322, 136],\n           dtype='int64', length=16). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"